Bu kursta, **veri sızıntısının** ne olduğunu ve nasıl önleneceğini öğreneceksiniz. Nasıl önleyeceğinizi bilmiyorsanız, sızıntı sık sık ortaya çıkar ve modellerinizi ince ve tehlikeli şekillerde mahveder. Dolayısıyla bu, veri bilimcileri için en önemli kavramlardan biridir.


# Giriş

**Veri sızıntısı** (veya **sızıntı**), eğitim verileriniz hedef hakkında bilgi içerdiğinde gerçekleşir, ancak model tahmin için kullanıldığında benzer veriler mevcut olmayacaktır. Bu, eğitim setinde (ve muhtemelen doğrulama verilerinde bile) yüksek performansa yol açar, ancak model üretimde düşük performans gösterecektir.

Başka bir deyişle, sızıntı, siz modelle karar vermeye başlayana kadar bir modelin doğru görünmesine neden olur ve ardından model çok yanlış hale gelir.

İki ana sızıntı türü vardır: **hedef sızıntı** ve **eğitim-doğrulama kontaminasyonu.**

### Hedef sızıntısı

**Hedef sızıntısı**, tahmincileriniz, tahminde bulunduğunuz sırada mevcut olmayacak verileri içerdiğinde meydana gelir. Hedef sızıntısı hakkında, yalnızca bir özelliğin iyi tahminler yapmaya yardımcı olup olmadığına değil, verilerin kullanılabilir hale geldiği _zamanlama veya kronolojik sıra_ açısından düşünmek önemlidir.

Bir örnek yardımcı olacaktır. Kimin zatürre ile hastalanacağını tahmin etmek istediğinizi hayal edin. Ham verilerinizin ilk birkaç satırı şöyle görünür:

| zatürresi_var | yaş |  kilo  | erkek |  antibiyotik_ilacı_aldı  | ... |
|:-------------:|:---:|:------:|:-----:|:------------------------:|-----|
|     False     |  65 |   100  | False |           False          | ... |
|     False     |  72 |   130  |  True |           False          | ... |
|      True     |  58 |   100  | False |           True           | ... |

İnsanlar zatürreye yakalandıktan *sonra* iyileşmek için antibiyotik ilaçlar alırlar. Ham veriler bu sütunlar arasında güçlü bir ilişki olduğunu gösterir, ancak `antibiyotik_ilacı_aldı` sık sık değiştirilir, `zatürresi_var` değeri belirlendikten sonra. Bu hedef sızıntısıdır.

Model, `antibiyotik_ilacı_aldı` için `False` değerine sahip herhangi birinin zatürre olmadığını görecekti. Doğrulama verileri eğitim verileriyle aynı kaynaktan geldiğinden, model doğrulamada kendini tekrar edecek ve model büyük doğrulama (veya çapraz doğrulama) puanlarına sahip olacaktır.

Ancak, gerçek dünyaya sonradan yerleştirildiğinde model çok yanlış olacaktır, çünkü zatürree olacak hastalar bile, gelecekteki sağlıkları hakkında tahminlerde bulunmamız gerektiğinde henüz antibiyotik almış olmayacaklardır.

Bu tür bir veri sızıntısını önlemek için, hedef değer gerçekleştikten sonra güncellenen (veya oluşturulan) herhangi bir değişken hariç tutulmalıdır.

### Eğitim-Doğrulama Kontaminasyonu

Eğitim verilerini doğrulama verilerinden ayırmaya dikkat etmediğinizde farklı bir sızıntı türü oluşur.

Doğrulamanın, modelin daha önce dikkate almadığı veriler üzerinde nasıl çalıştığının bir ölçüsü olması gerektiğini hatırlayın. Doğrulama verileri ön işleme davranışını etkiliyorsa, bu işlemi ince yöntemlerle bozabilirsiniz. Buna bazen **eğitim-doğrulama kontaminasyonu** denir.

Örneğin, [`train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) öğesini çağırmadan önce ön işleme (eksik değerler için bir imputer takmak gibi) yaptığınızı düşünün. . Sonuç? Modeliniz iyi doğrulama puanları alabilir, ancak karar vermek için onu dağıttığınızda düşük performans gösterir.

Ne de olsa, doğrulama veya test verilerinden gelen verileri tahminlerinizi nasıl yaptığınıza dahil ettiniz, böylece yeni verilere genelleme yapamasa bile bu belirli veriler üzerinde iyi sonuç verebilir.

Doğrulamanız basit bir train-test bölünmesine dayanıyorsa, ön işleme adımlarının uyumu da dahil olmak üzere herhangi bir *fit etme* türünden doğrulama verilerini hariç tutun. Scikit-learn boru hatları kullanıyorsanız bu daha kolaydır. Çapraz doğrulama kullanırken, ön işlemenizi boru hattı içinde yapmanız daha da kritiktir!

# Örnek

Bu örnekte, hedef sızıntıyı tespit etmenin ve ortadan kaldırmanın bir yolunu öğreneceksiniz.

Kredi kartı uygulamaları hakkında bir veri seti kullanacağız ve temel veri kurulum kodunu atlayacağız. Sonuç olarak, her bir kredi kartı uygulamasıyla ilgili bilgiler bir DataFrame `X`içinde saklanır. Bunu, `y` Serisinde hangi başvuruların kabul edildiğini tahmin etmek için kullanacağız. 

In [ ]:
import pandas as pd

# Veriyi oku
data = pd.read_csv('../input/aer-credit-card-data/AER_credit_card_data.csv', 
                   true_values = ['yes'], false_values = ['no'])

# Hedefi seç
y = data.card

# Tahmincileri seç
X = data.drop(['card'], axis=1)

print("Veri kümesi içindeki satırların sayısı:", X.shape[0])
X.head()

Bu küçük bir veri kümesi olduğundan, model kalitesinin doğru ölçümlerini sağlamak için çapraz doğrulama kullanacağız. 

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Ön işleme olmadığından, bir boru hattına ihtiyacımız yok
my_pipeline = make_pipeline(RandomForestClassifier(n_estimators=100))
cv_scores = cross_val_score(my_pipeline, X, y, 
                            cv=5,
                            scoring='accuracy')

print("Çapraz doğrulama doğruluğu : %f" % cv_scores.mean())

Deneyimle, %98 doğruluğa sahip modeller bulmanın çok nadir olduğunu göreceksiniz. Bu olur, ancak hedef sızıntısı için verileri daha yakından incelememiz yeterince nadirdir.

Veri sekmesi altında da bulabileceğiniz verilerin bir özeti:

 - **`card`**: Kredi kartı başvurusu kabul edilirse 1, kabul edilmezse 0
 - **`reports`**: Büyük aşağılayıcı raporların sayısı
 - **`age`**: Yaş n yıl artı bir yılın on ikide biri
 - **`income`**: Yıllık gelir (10.000'e bölünür)
 - **`share`**: Aylık kredi kartı harcamalarının yıllık gelire oranı
 - **`expenditure`**: Ortalama aylık kredi kartı harcaması
 - **`owner`**: Ev sahibi ise 1, kira ise 0
 - **`selfempl`**: Serbest meslek sahibi ise 1, değilse 0
 - **`dependents`**: 1 + bağımlı sayısı
 - **`months`**: Geçerli adreste yaşanan aylar
 - **`majorcards`**: Sahip olunan başlıca kredi kartlarının sayısı
 - **`active`**: Aktif kredi hesaplarının sayısı

Birkaç değişken şüpheli görünüyor. Örneğin, **`expenditure`** bu karttan mı yoksa başvuru öncesi kullanılan kartlardan yapılan harcama anlamına mı geliyor?

Bu noktada, temel veri karşılaştırmaları çok yardımcı olabilir: 

In [ ]:
expenditures_cardholders = X.expenditure[y]
expenditures_noncardholders = X.expenditure[~y]

print('Kart almayan ve harcama yapmayanların oranı: %.2f' \
      %((expenditures_noncardholders == 0).mean()))
print('Kart alıp harcama yapmayanların oranı: %.2f'  \
      %(( expenditures_cardholders == 0).mean()))

Yukarıda da görüldüğü gibi, kart almayan herkesin harcaması yokken, kart alanların sadece %2'sinin harcaması yok. Modelimizin yüksek bir doğruluğa sahip görünmesi şaşırtıcı değil. Ancak bu aynı zamanda harcamaların muhtemelen *başvurdukları karttaki harcamalar* anlamına geldiği bir hedef kaçağı durumu gibi görünüyor.

**`share`** kısmen **`expenditures`** tarafından belirlendiğinden, bunun da hariç tutulması gerekir. **`active`** ve **`major cards`** değişkenleri biraz daha az anlaşılır, ancak açıklamadan endişe verici görünüyorlar. Çoğu durumda, daha fazlasını öğrenmek için verileri oluşturan kişilerin izini bulamıyorsanız, üzülmektense güvende olmak daha iyidir.

Hedef sızıntısı olmayan bir modeli aşağıdaki gibi çalıştırırız: 

In [ ]:
# Veri kümesinden sızdıran tahmincileri düşürün 
potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(potential_leaks, axis=1)

# Sızdıran tahmin ediciler kaldırılmış olarak modeli değerlendirin 
cv_scores = cross_val_score(my_pipeline, X2, y, 
                            cv=5,
                            scoring='accuracy')

print("Çapraz doğrulama doğruluğu : %f" % cv_scores.mean())

Bu doğruluk biraz daha düşüktür, bu da hayal kırıklığı yaratabilir. Bununla birlikte, yeni uygulamalarda kullanıldığında zamanın yaklaşık %80'inde doğru olmasını bekleyebiliriz, oysa sızdıran model muhtemelen bundan çok daha kötü sonuç verecektir (çapraz doğrulamada daha yüksek görünen puanına rağmen).

# Sonuç
Veri sızıntısı, birçok veri bilimi uygulamasında milyonlarca dolarlık bir hata olabilir. Eğitim ve doğrulama verilerinin dikkatli bir şekilde ayrılması, eğitim-doğrulama kontaminasyonunu önleyebilir ve boru hatları bu ayrımın uygulanmasına yardımcı olabilir. Benzer şekilde, dikkatli olma, sağduyu ve veri keşfinin bir kombinasyonu, hedef sızıntının belirlenmesine yardımcı olabilir. 

# Tebrikler!

Orta düzey makine öğrenimi kursunun sonuna geldiniz. Artık son derece hassas modeller yapmak için araçlara sahipsiniz ve bu modelleri gerçek sorunları çözmek için uygularken ortaya çıkan en zor pratik sorunları çözebilirsiniz. Neler öğrendiğinize göz atmak için **[Veri bilimi 101 Türkçe](https://www.kaggle.com/bolsogansiz/veri-bilimi-101-t-rk-e)** kurs serisinin ana sayfasına gidebilirsiniz. 


Tekrar, Tebrikler! 